### Upgrade/install necessary packages

In [101]:
!pip install --upgrade --user ortools
!pip install altair

Requirement already up-to-date: ortools in /home/azureuser/.local/lib/python3.8/site-packages (9.4.1874)
     |████████████████████████████████| 812 kB 7.6 MB/s eta 0:00:01


### Adapted Job Shop example from OR-Tools

In [1]:
"""Minimal jobshop example."""
import collections
from ortools.sat.python import cp_model


def generate_schedule(jobs_data):
    """Minimal jobshop problem."""

    machines_count = 1 + max(task[0] for job in jobs_data for task in job)
    all_machines = range(machines_count)
    # Computes horizon dynamically as the sum of all durations.
    horizon = sum(task[1] for job in jobs_data for task in job)

    # Create the model.
    model = cp_model.CpModel()

    # Named tuple to store information about created variables.
    task_type = collections.namedtuple('task_type', 'start end interval min_id')
    # Named tuple to manipulate solution information.
    assigned_task_type = collections.namedtuple('assigned_task_type',
                                                'start job index duration min_id')

    # Creates job intervals and add to the corresponding machine lists.
    all_tasks = {}
    machine_to_intervals = collections.defaultdict(list)

    for job_id, job in enumerate(jobs_data):
        for task_id, task in enumerate(job):
            machine = task[0]
            duration = task[1]
            min_id = task[2]
            suffix = '_%i_%i' % (job_id, task_id)
            start_var = model.NewIntVar(0, horizon, 'start' + suffix)
            end_var = model.NewIntVar(0, horizon, 'end' + suffix)
            interval_var = model.NewIntervalVar(start_var, duration, end_var,
                                                'interval' + suffix)
            all_tasks[job_id, task_id] = task_type(start=start_var,
                                                   end=end_var,
                                                   interval=interval_var, min_id=min_id)
            machine_to_intervals[machine].append(interval_var)

    # Create and add disjunctive constraints.
    for machine in all_machines:
        model.AddNoOverlap(machine_to_intervals[machine])

    # Precedences inside a job.
    for job_id, job in enumerate(jobs_data):
        for task_id in range(len(job) - 1):
            
            # Adaptation to allow offsets in start of UF during execution of R368 
            end_offset = int(all_tasks[job_id, task_id].interval.SizeExpr() * .333)
            model.Add(all_tasks[job_id, task_id +
                                1].start >= all_tasks[job_id, task_id].end - end_offset)
            

    # Makespan objective.
    obj_var = model.NewIntVar(0, horizon, 'makespan')
    model.AddMaxEquality(obj_var, [
        all_tasks[job_id, len(job) - 1].end
        for job_id, job in enumerate(jobs_data)
    ])
    model.Minimize(obj_var)

    # Creates the solver and solve.
    solver = cp_model.CpSolver()
    status = solver.Solve(model)

    if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
        print('Solution:')
        # Create one list of assigned tasks per machine.
        assigned_jobs = collections.defaultdict(list)
        for job_id, job in enumerate(jobs_data):
            for task_id, task in enumerate(job):
                machine = task[0]
                min_id = task[2]
                assigned_jobs[machine].append(
                    assigned_task_type(start=solver.Value(
                        all_tasks[job_id, task_id].start),
                                       job=job_id,
                                       index=task_id,
                                       duration=task[1], min_id=min_id))

        # Create per machine output lines.
        output = ''
        return_dict = {}
        for machine in all_machines:
            machine_name = ''
            if machine==0:
                machine_name = 'Reactor 368'
            if machine==1:
                machine_name = 'UF'
            # Sort by starting time.
            assigned_jobs[machine].sort()
            sol_line_tasks = 'Machine ' + str(machine) + ': '
            sol_line = '           '

            for assigned_task in assigned_jobs[machine]:
                if assigned_task.min_id not in return_dict.keys():
                    return_dict[assigned_task.min_id] = {}
                if machine_name not in return_dict[assigned_task.min_id].keys():
                    return_dict[assigned_task.min_id][machine_name] = {}
                    
                return_dict[assigned_task.min_id][machine_name][assigned_task.job] = [assigned_task.start, assigned_task.start + assigned_task.duration]
                name = assigned_task.min_id + '_'
                name +=  '%i_%i' % (assigned_task.job,
                                           assigned_task.index)
                # Add spaces to output to align columns.
                sol_line_tasks += '%-15s' % name

                start = assigned_task.start
                duration = assigned_task.duration
                sol_tmp = '[%i,%i]' % (start, start + duration)
                # Add spaces to output to align columns.
                sol_line += '%-15s' % sol_tmp

            sol_line += '\n'
            sol_line_tasks += '\n'
            output += sol_line_tasks
            output += sol_line

        # Finally print the solution found.
        print(f'Optimal Schedule Length: {solver.ObjectiveValue()}')
        print(output)
    else:
        print('No solution found.')

    # Statistics.
    print('\nStatistics')
    print('  - conflicts: %i' % solver.NumConflicts())
    print('  - branches : %i' % solver.NumBranches())
    print('  - wall time: %f s' % solver.WallTime())
    
    return return_dict


In [2]:
mo7a5_reqd_batches = 11
bo5y5_reqd_batches = 3

mo7a5_def = [(0, 40, 'MO7A5'), (1, 16, 'MO7A5')]
b05y5_def = [(0, 10, 'BO5Y5'), (1, 8, 'BO5Y5')]
jobs_data = []

for i in range(mo7a5_reqd_batches):
    jobs_data.append(mo7a5_def)
for i in range (bo5y5_reqd_batches):
    jobs_data.append(b05y5_def)

out = generate_schedule(jobs_data)

out

Solution:
Optimal Schedule Length: 473.0
Machine 0: BO5Y5_11_0     BO5Y5_12_0     BO5Y5_13_0     MO7A5_0_0      MO7A5_1_0      MO7A5_2_0      MO7A5_3_0      MO7A5_4_0      MO7A5_5_0      MO7A5_6_0      MO7A5_7_0      MO7A5_8_0      MO7A5_9_0      MO7A5_10_0     
           [0,10]         [10,20]        [20,30]        [30,70]        [70,110]       [110,150]      [150,190]      [190,230]      [230,270]      [270,310]      [310,350]      [350,390]      [390,430]      [430,470]      
Machine 1: BO5Y5_11_1     BO5Y5_12_1     BO5Y5_13_1     MO7A5_0_1      MO7A5_1_1      MO7A5_2_1      MO7A5_3_1      MO7A5_4_1      MO7A5_5_1      MO7A5_6_1      MO7A5_7_1      MO7A5_8_1      MO7A5_9_1      MO7A5_10_1     
           [7,15]         [17,25]        [27,35]        [57,73]        [97,113]       [137,153]      [177,193]      [217,233]      [257,273]      [297,313]      [337,353]      [377,393]      [417,433]      [457,473]      


Statistics
  - conflicts: 0
  - branches : 0
  - wall time: 0.010844 

{'BO5Y5': {'Reactor 368': {11: [0, 10], 12: [10, 20], 13: [20, 30]},
  'UF': {11: [7, 15], 12: [17, 25], 13: [27, 35]}},
 'MO7A5': {'Reactor 368': {0: [30, 70],
   1: [70, 110],
   2: [110, 150],
   3: [150, 190],
   4: [190, 230],
   5: [230, 270],
   6: [270, 310],
   7: [310, 350],
   8: [350, 390],
   9: [390, 430],
   10: [430, 470]},
  'UF': {0: [57, 73],
   1: [97, 113],
   2: [137, 153],
   3: [177, 193],
   4: [217, 233],
   5: [257, 273],
   6: [297, 313],
   7: [337, 353],
   8: [377, 393],
   9: [417, 433],
   10: [457, 473]}}}

In [3]:
from datetime import datetime, timedelta
start_time = datetime(2022, 11, 1)

import altair as alt
import datetime as dt
import pandas as pd


alt.renderers.enable('jupyterlab')

data = pd.DataFrame()
from_data = []
to_data = []
activity_data = []


for min_id, mach_vals in out.items():
    for mach_id, job_vals in mach_vals.items():
        for job_id, job_interval in job_vals.items():
            activity = f'{min_id} - {mach_id}'
            begin = start_time + timedelta(hours = job_interval[0])
            end = start_time + timedelta(hours = job_interval[1]-.25)
            from_data.append((begin))
            to_data.append((end))
            activity_data.append(activity)
            
data['Start Time'] = from_data
data['End Time'] = to_data
data['Processing Activity'] = activity_data


alt.Chart(data).mark_bar().encode(
    x='Start Time',
    x2='End Time',
    y='Processing Activity',
    color=alt.Color('Processing Activity', scale=alt.Scale(scheme='dark2'))
).properties(
    width=800,
    height=300
)

<VegaLite 4 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html
